In [1]:
import glob
import argparse
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import oggm
from oggm import utils

from __future__ import print_function, division   # Ensures Python3 printing & division standard
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.pylab import rcParams

import numpy as np
import pandas as pd 
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score,mean_squared_error, mean_absolute_error, r2_score, roc_curve,auc
from sklearn.tree import DecisionTreeClassifier, plot_tree,export_graphviz
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier

from scipy.stats import randint, poisson
from sklearn.preprocessing import MinMaxScaler

import xgboost as xgb
from xgboost import plot_tree

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

from ipywidgets import interactive
from graphviz import Source
from IPython.display import SVG

import csv

%matplotlib qt 

In [3]:
# Setup oggm: important we want to use oggm's version 62 of all glacier geometries. This command will download them.
utils.get_rgi_dir(version='62')

'C:\\Users\\damsg\\OGGM\\rgi\\RGIV62'

In [3]:
path_O1_shp = '00_rgi60_O1Regions.shp' # shp file of regional boundaries

# Import 20-bin gridded training dataset
metadata_file = "metadata19_hmineq0.0_tmin20050000_mean_grid_20.csv"
glathida_rgis = pd.read_csv(metadata_file, low_memory=False)

In [27]:
X = glathida_rgis.drop(['ith_m','ith_f','THICKNESS','RGIId'],axis=1) 
scaler = MinMaxScaler()

X = pd.DataFrame(scaler.fit_transform(X.values), columns=X.columns, index=X.index)
y = glathida_rgis['THICKNESS'] 

X_train, X_test, y_train, y_test = train_test_split(X, y,train_size = 0.95, random_state=1)

model = Sequential([
    Dense(50,activation='LeakyReLU'   ,name='input_layer'),
    Dense(75,activation='relu'   ,name='hidden_layer1'),
    Dense(75,activation='relu'   ,name='hidden_layer2'),
    Dense(1, name='output')])

model.compile(optimizer='adam',
              loss= 'mean_absolute_error',
              metrics=['mean_absolute_error'])

print('--------- TRAINING ---------')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(x = np.array(X_train), y = np.array(y_train),validation_data=(np.array(X_test), np.array(y_test)), 
                    epochs = 50, callbacks=[early_stopping]) 

--------- TRAINING ---------
Epoch 1/50
2414/2414 [==============================] - 5s 2ms/step - loss: 74.8508 - mean_absolute_error: 74.8508 - val_loss: 65.9274 - val_mean_absolute_error: 65.9274
Epoch 2/50
2414/2414 [==============================] - 4s 2ms/step - loss: 64.9389 - mean_absolute_error: 64.9389 - val_loss: 62.6031 - val_mean_absolute_error: 62.6031
Epoch 3/50
2414/2414 [==============================] - 5s 2ms/step - loss: 61.7836 - mean_absolute_error: 61.7836 - val_loss: 61.7823 - val_mean_absolute_error: 61.7823
Epoch 4/50
2414/2414 [==============================] - 5s 2ms/step - loss: 59.3155 - mean_absolute_error: 59.3155 - val_loss: 57.0590 - val_mean_absolute_error: 57.0590
Epoch 5/50
2414/2414 [==============================] - 4s 2ms/step - loss: 57.8678 - mean_absolute_error: 57.8678 - val_loss: 57.2344 - val_mean_absolute_error: 57.2344
Epoch 6/50
2414/2414 [==============================] - 4s 2ms/step - loss: 57.1195 - mean_absolute_error: 57.1195 - val_

In [19]:
# print(X_test)

In [29]:
preds = model.predict(X_test)
r2_score(preds, y_test)

128/128 [==============================] - 0s 1ms/step


0.6690484659870387

In [42]:
# See all RGI's regional boundaries
# See pag 8 at https://nsidc.org/sites/nsidc.org/files/technical-references/RGI_Tech_Report_V6.0.pdf
world = gpd.read_file(path_O1_shp)      # import as geopandas dataframe
print(world)
fig, ax = plt.subplots()
for n, rgi_series in enumerate(world['geometry']):
    x, y = rgi_series.exterior.xy
    min_x, max_x = min(x), max(x)
    min_y, max_y = min(y), max(y)
    ax.plot(x, y, c='g')
    # ax.text(np.mean([min_x, max_x]), np.mean([min_y, max_y]), f"{world['RGI_CODE'].iloc[n]}", color='green', fontsize=9)
    ax.text(np.mean([min_x, max_x]), np.mean([min_y, max_y]), f"{world['geometry'].iloc[n]}", color='green', fontsize=9)

plt.show()

                                             geometry
0   POLYGON ((-133.00000 54.50000, -134.00000 54.5...
1   POLYGON ((180.00000 50.00000, 179.00000 50.000...
2   POLYGON ((-133.00000 54.50000, -132.00000 54.5...
3   POLYGON ((-125.00000 74.00000, -125.00000 75.0...
4   POLYGON ((-90.00000 74.00000, -89.00000 74.000...
5   POLYGON ((-75.00000 77.00000, -74.73000 77.510...
6   POLYGON ((-26.00000 59.00000, -26.00000 60.000...
7   POLYGON ((-10.00000 70.00000, -10.00000 71.000...
8   POLYGON ((4.00000 70.00000, 4.00000 71.00000, ...
9   POLYGON ((35.00000 70.00000, 35.00000 71.00000...
10  POLYGON ((-180.00000 78.00000, -179.00000 78.0...
11  POLYGON ((128.00000 46.00000, 127.00000 46.000...
12  POLYGON ((-6.00000 40.00000, -6.00000 41.00000...
13  POLYGON ((32.00000 31.00000, 32.00000 32.00000...
14  POLYGON ((80.00000 46.00000, 81.00000 46.00000...
15  POLYGON ((75.40000 26.00000, 75.00000 26.00000...
16  POLYGON ((75.40000 26.00000, 75.40000 27.00000...
17  POLYGON ((-100.00000 -25

In [69]:
# Let's select one region of interest, e.g. rgi=11 (Central Europe)

# Dataframe of glaciers for rgi=11. This dataset contains a lot of info, some of them are also used to create the
# training dataset. RGIId is the official name of glaciers according to the Randolph Glacier Inventory. 
# In the column geometry you get the geometries. 

rgi = 11
oggm_rgi_shp = "11_rgi62_CentralEurope.shp" # .shp file for rgi 11
# oggm_rgi_shp = glob.glob(f"/home/maffe/OGGM/rgi/RGIV62/{rgi}*/{rgi}*.shp")[0] # .shp file for rgi 11
oggm_rgi_glaciers = gpd.read_file(oggm_rgi_shp)                         # dataframe for rgi 11


In [81]:
oggm_rgi_glaciers['geometry']


0       POLYGON ((13.60035 47.49330, 13.59995 47.49332...
1       POLYGON ((13.60638 47.47578, 13.60599 47.47579...
2       POLYGON ((13.59765 47.47613, 13.59726 47.47614...
3       POLYGON ((13.58283 47.47969, 13.58243 47.47971...
4       POLYGON ((13.60076 47.47519, 13.60036 47.47521...
                              ...                        
3922    POLYGON ((13.44285 46.36257, 13.44293 46.36257...
3923    POLYGON ((13.44437 46.43624, 13.44404 46.43624...
3924    POLYGON ((13.44764 46.36464, 13.44761 46.36465...
3925    POLYGON ((13.44990 46.36638, 13.44989 46.36644...
3926    POLYGON ((13.47392 46.36846, 13.47396 46.36846...
Name: geometry, Length: 3927, dtype: geometry

In [86]:
# Plot one specific glacier by passing the RGIId code.
# Example: RGI60-11.01450 is the Aletsch Glacier, the biggest in Central Europe (rgi=11).
# You under the geometry column you find the glacier external boundary (.external)
# and the glacier nunataks, if any (.interiors). Nunataks are the inner glacier portions that are deglaciated (=rock).
# We can also find the points in the training dataset for this glacier (not all glaciers contain measurements)

# glacier_geometry = oggm_rgi_glaciers.loc[oggm_rgi_glaciers['RGIId']=='RGI60-11.01450']['geometry'].item()

# Get the measurements in the training dataset by passing the glacier id
glathida_rgis_aletsch = glathida_rgis.loc[glathida_rgis['RGIId']=='RGI60-11.01450']
print(f"We have {len(glathida_rgis_aletsch)} points in the training dataset")

fig, ax = plt.subplots()
exterior_ring = glacier_geometry.exterior # External geometry
ax.plot(*exterior_ring.xy, c='b')
glacier_nunataks_list = [nunatak for nunatak in glacier_geometry.interiors] # Nunataks (list may be empty if no nunataks)
for nunatak in glacier_nunataks_list:
    ax.plot(*nunatak.xy, c='k', lw=0.8)
ax.scatter(x=glathida_rgis_aletsch['POINT_LON'], y=glathida_rgis_aletsch['POINT_LAT'], s=20, c='orange', label='Training points')
ax.legend()
plt.show()

We have 49 points in the training dataset


NameError: name 'glacier_geometry' is not defined

In [ ]:
# Plot the whole regional set of glaciers and all nunataks
# zoom in with matplotlib in interactive mode (%matplotlib qt ) to have a sense of all glacier and their nunataks

region11 = world.loc[12] # Regional boundary
fig, ax = plt.subplots()
for glacier_rgiid, glacier_geometry in zip(oggm_rgi_glaciers['RGIId'], oggm_rgi_glaciers['geometry']):

    exterior_ring = glacier_geometry.exterior
    glacier_nunataks_list = [nunatak for nunatak in glacier_geometry.interiors]

    ax.plot(*exterior_ring.xy, c='b')
    for nunatak in glacier_nunataks_list:
        ax.plot(*nunatak.xy, c='k', lw=0.8) # Plot glacier nunataks (if any)
ax.plot(*region11['geometry'].exterior.xy, c='g') # Regional boundary
ax.text(min(*region11['geometry'].exterior.xy[0])+1,
        max(*region11['geometry'].exterior.xy[1])-1, f'rgi {rgi}', c='g', fontsize=12)
plt.show()